# VOCS data structure 

In [1]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == ObjectiveEnum.MAXIMIZE

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

In [10]:
vocs.schema()

{'title': 'VOCS',
 'description': 'Variables, Objectives, Constraints, and other Settings (VOCS) data structure\nto describe optimization problems.',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'default': {},
   'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'default': {}, 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'additionalProperties': Fal

# Objective Evaluation

In [11]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,899.940251,0.556665,1234,900.496916,899.383587,1800.993832,1798.767173
6,41.977861,-0.615481,1234,41.362380,42.593341,82.724759,85.186683
7,966.342098,0.809863,1234,967.151961,965.532235,1934.303922,1931.064470
8,292.487664,-0.804832,1234,291.682832,293.292497,583.365665,586.584993
9,648.519754,0.828434,1234,649.348188,647.691320,1298.696377,1295.382641
10,459.060711,-0.351481,1234,458.709230,459.412191,917.418460,918.824383
11,19.894925,-0.847953,1234,19.046972,20.742879,38.093944,41.485757
12,84.910553,-0.970707,1234,83.939847,85.881260,167.879693,171.762520
13,796.510290,-0.908380,1234,795.601910,797.418670,1591.203821,1594.837341
14,5.887745,0.545660,1234,6.433405,5.342085,12.866810,10.684169


In [12]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [13]:
?form_objective_data

Signature: form_objective_data(objectives: Dict, data, prefix='objective_')
Docstring:
Use objective dict and data (dataframe) to generate objective data (dataframe)

Weights are applied to convert all objectives into mimimization form.

Returns a dataframe with the objective data intented to be minimized.

Missing or nan values will be filled with: np.inf
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [14]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-900.496916,899.383587
6,-41.362380,42.593341
7,-967.151961,965.532235
8,-291.682832,293.292497
9,-649.348188,647.691320
10,-458.709230,459.412191
11,-19.046972,20.742879
12,-83.939847,85.881260
13,-795.601910,797.418670
14,-6.433405,5.342085


In [15]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-900.496916,899.383587
6,-41.362380,42.593341
7,-967.151961,965.532235
8,-291.682832,293.292497
9,-649.348188,647.691320
10,-458.709230,459.412191
11,-19.046972,20.742879
12,-83.939847,85.881260
13,-795.601910,797.418670
14,-6.433405,5.342085


In [16]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-900.49691616,  899.38358653],
       [ -41.36237972,   42.59334143],
       [-967.151961  ,  965.53223507],
       [-291.68283234,  293.29249653],
       [-649.34818829,  647.6913204 ],
       [-458.70922981,  459.41219149],
       [ -19.046972  ,   20.74287851],
       [ -83.93984657,   85.88125991],
       [-795.60191039,  797.4186705 ],
       [  -6.43340493,    5.34208451]])

In [17]:
?form_constraint_data

Signature:
form_constraint_data(
    constraints: Dict,
    data: pandas.core.frame.DataFrame,
    prefix='constraint_',
)
Docstring:
Use constraint dict and data (dataframe) to generate constraint data (dataframe). A
constraint is satisfied if the evaluation is < 0.

Args:
    constraints: Dictonary of constraints
    data: Dataframe with the data to be evaluated
    prefix: Prefix to use for the transformed data in the dataframe

Returns a dataframe with the constraint data.

Missing or nan values will be filled with: np.inf
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [18]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1798.993832,-1798.767173
6,80.724759,-85.186683
7,1932.303922,-1931.064470
8,581.365665,-586.584993
9,1296.696377,-1295.382641
10,915.418460,-918.824383
11,36.093944,-41.485757
12,165.879693,-171.762520
13,1589.203821,-1594.837341
14,10.866810,-10.684169


In [19]:
?form_feasibility_data

Signature: form_feasibility_data(constraints: Dict, data, prefix='feasible_')
Docstring:
Use constraint dict and data to identify feasible points in the the dataset.

Returns a dataframe with the feasibility data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [20]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [21]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [22]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [23]:
data['a']  = np.nan

In [24]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [25]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [26]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
